# Machine Learning con Python

## 1. Cargamos los datos en Python

In [1]:
import pyarrow.feather as feather
import pandas as pd
import numpy as np
disp_df = pd.read_feather("disp_9.feather")
disp_df.head()

,salida,apcp_sf1_1,apcp_sf2_1,apcp_sf3_1,dlwrf_s1_1,dlwrf_s4_1,dswrf_s1_1,dswrf_s2_1,dswrf_s4_1,dswrf_s5_1,...,ulwrf_s2_1,ulwrf_s4_1,uswrf_s1_1,uswrf_s2_1,uswrf_s3_1,dlwrf_s2_1,dlwrf_s3_1,dlwrf_s5_1,tmp_2m_4_1,ulwrf_t4_1
0,10848300.0,1,1,1,2,1,1,1,2,0.0,...,328.249736,376.864962,0.000000,7.000000,45.000000,red,red,red,blue,blue
1,3318300.0,1,1,1,2,2,1,1,1,0.0,...,328.718309,380.017767,0.000000,2.181818,29.181818,blue,blue,blue,blue,blue
2,5266500.0,1,1,1,1,1,1,1,2,0.0,...,288.570143,354.793682,0.000000,8.181818,45.636364,red,red,red,red,blue
3,8270100.0,1,1,1,1,1,1,1,2,0.0,...,305.488836,352.025060,0.124633,5.272727,37.000000,blue,blue,red,red,blue
4,8009400.0,1,1,1,1,2,1,1,2,0.0,...,374.605434,374.737835,0.000000,6.454545,42.272727,red,red,blue,blue,blue


## 2. Aprendizaje Automático con sklearn

### 2.1 EDA

In [2]:
print('The shape of the data table is:')
print('===============================')
print(disp_df.shape)
print()

print('The types of the attributes are:')
print('================================')
disp_df.info()

print()

print('Fraction of missing values per attribute:')
print('======================================')
print(disp_df.isnull().mean())

The shape of the data table is:
(4380, 76)

The types of the attributes are:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4380 entries, 0 to 4379
Data columns (total 76 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   salida      4380 non-null   float64 
 1   apcp_sf1_1  4380 non-null   int32   
 2   apcp_sf2_1  4380 non-null   int32   
 3   apcp_sf3_1  4380 non-null   int32   
 4   dlwrf_s1_1  4380 non-null   int32   
 5   dlwrf_s4_1  4380 non-null   int32   
 6   dswrf_s1_1  4380 non-null   int32   
 7   dswrf_s2_1  4380 non-null   int32   
 8   dswrf_s4_1  4380 non-null   int32   
 9   dswrf_s5_1  4380 non-null   float64 
 10  pres_ms1_1  4380 non-null   int32   
 11  pres_ms2_1  4380 non-null   int32   
 12  pres_ms3_1  4380 non-null   int32   
 13  pres_ms4_1  4380 non-null   int32   
 14  pwat_ea1_1  4380 non-null   float64 
 15  pwat_ea2_1  4380 non-null   int32   
 16  pwat_ea3_1  4380 non-null   float64 
 17  pwat_ea4_1  4

Como podemos observar, hay 9 variables categóricas de las 75 variables que hay. Y comprobamos que tampoco hay missing values.

### 2.2 Matrices y creación de dummies

In [19]:
y = disp_df['salida'].values


X_df = disp_df.drop('salida', axis=1)


from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector

preprocessor = ColumnTransformer(
    transformers = [ 
                    ('categorical', OneHotEncoder(handle_unknown='ignore'),  selector(dtype_include="category"))
                    ], 
                    remainder='passthrough' 
)

preprocessor.fit(X_df)
X = preprocessor.transform(X_df)

print(X.shape)
print()

print(type(X))
print()

print(X)

(4380, 93)

<class 'numpy.ndarray'>

[[ 0.          0.          1.         ...  0.          7.
  45.        ]
 [ 0.          0.          1.         ...  0.          2.18181818
  29.18181818]
 [ 0.          0.          1.         ...  0.          8.18181818
  45.63636364]
 ...
 [ 0.          0.          1.         ...  0.          6.27272727
  76.78435629]
 [ 0.          0.          1.         ...  0.          7.72727273
  47.45454545]
 [ 0.          0.          1.         ...  0.          6.63636364
  43.72727273]]


Hemos usado ColumnTransformer para crear dummies con las 9 variables categóricas que quedaban. Podemos ver como ahora disponemos de 93 variables, de las cuales 27 son dummies y las 63 restantes que no son categóricas.
Además, previamente hemos creado la matriz 'X' con los predictores y la matriz 'y' con la variable respuesta 'salida', ya que sklearn no sabe trabajar con data.frames.

### 2.3 Árbol de regresión

In [26]:
from sklearn import metrics
from sklearn import tree


X_train = X[0:(6*365-1),:]
X_test = X[(6*365):(9*365-1),:]
y_train = y[0:(6*365-1)]
y_test = y[(6*365):(9*365-1)]


regr = tree.DecisionTreeRegressor()
np.random.seed(100430523) 

regr.fit(X_train, y_train)

y_test_pred = regr.predict(X_test)

mae_validation_tree = metrics.mean_absolute_error(y_test, y_test_pred)
print(f"MAE of the tree: {mae_validation_tree}")

MAE of the tree: 3532477.970749543


Para la muestra de entrenamiento, vamos a usar tal y como usamos en las otras prácticas, los 6 primeros años; y para la muestra test o de validation, los 3 años siguientes.
Por tanto, el MAE (metric.mean_absolute_error) del árbol de regresión es de 3532477.97.

### 2.4 KNN

In [27]:
from sklearn.neighbors import KNeighborsRegressor as KNN
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

scaler = StandardScaler()
knn = KNN()

pipe_regr = Pipeline([
    ('scale', scaler),
    ('KNN', knn)])

np.random.seed(100430523)
pipe_regr.fit(X=X_train, y=y_train)
mae_validation_knn = metrics.mean_absolute_error(y_test, pipe_regr.predict(X=X_test))
print(f"MAE of KNN with default hyper-pars: {mae_validation_knn}")


MAE of KNN with default hyper-pars: 3009033.5356489946


Para aplicar el método KNN, hemos tenido que escalar los datos.
El MAE (metric.mean_absolute_error) de KNN sin ajuste de hiper-parámetros es de 3009033.53, algo menor que el del árbol de regresión.

### 2.5 COMPARACIÓN DE RAE (sklearn VS mlr3)

In [28]:
mae_validation_dummy = np.mean(np.abs(y_test-y_train.mean()))
rae_validation_tree = mae_validation_tree / mae_validation_dummy
print(f"RAE of the tree: {rae_validation_tree}")

rae_validation_knn = mae_validation_knn / mae_validation_dummy
print(f"RAE of the Knn: {rae_validation_knn}")

RAE of the tree: 0.5287467549910644
RAE of the Knn: 0.45039678401620814


A partir del MAE del modelo entre el MAE de un modelo trivial, en este caso hemos usado la media, se calcula el RAE (Relative Absolute Error).

**El RAE del árbol de regresión es: 0.5287**

**El RAE de KNN con ajuste de hiper-parámetros es: 0.4503**

Comparando los RAE que nos salen en esta práctica con sklearn y los que nos salían con mlr3, observamos claramente que los obtenidos con **sklearn** son un poco más altos respecto a todos los de mlr3 que se hallaban más cerca del 0.4. Esto tiene sentido debido a que en mlr3 escogimos los métodos de imputación y escalado óptimos para nuestros datos y esta vez hemos cogido unos diferentes.